In [404]:
df_filtered['cnt'] = df_filtered['description'].apply(lambda x: len(str(x).split()))

In [405]:
df = df_filtered[df_filtered['cnt']<250]

**Extract

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('before_extract/techer_uni.csv')

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup as bs

def preprocess(sentence):
    soup = bs(sentence, features="html.parser")
    sentence = soup.get_text()
    
    sentence = str(sentence).lower()
    sentence = sentence.replace('{html}',"")
    
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url = re.sub(r'http\S+', '', cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    
    tokenizer = RegexpTokenizer(r'\w+|[.,]')
    tokens = tokenizer.tokenize(rem_num)
    
    filtered_words = [w for w in tokens if not w in stopwords.words('russian')]

    processed_text = re.sub(r'\s+([.,])', r'\1', " ".join(filtered_words))
    
    processed_text = re.sub(r'\.\s*\.', '.', processed_text)
    
    return processed_text

In [9]:
df['extr'] = df['description'].progress_apply(preprocess)

  2%|▏         | 441/17685 [00:06<03:36, 79.68it/s]/tmp/ipykernel_30458/476461610.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")
100%|██████████| 17685/17685 [05:11<00:00, 56.71it/s]


In [10]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

data = df  
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def summarize_text(text, num_sentences=4):
    sentences = sent_tokenize(text, language='russian')
    
    sentence_embeddings = model.encode(sentences)
    
    similarity_matrix = cosine_similarity(sentence_embeddings)
    
    sentence_scores = similarity_matrix.sum(axis=1)
    
    top_sentence_indices = np.argsort(sentence_scores)[-num_sentences:]
    
    top_sentence_indices = sorted(top_sentence_indices)
    
    summary_sentences = [sentences[i].strip() for i in top_sentence_indices if sentences[i].strip()]
    summary = " ".join(summary_sentences)
    
    return summary

tqdm.pandas()

/opt/jupyterhub/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to
[nltk_data]     /home/maulen_auth0_auth0/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/opt/jupyterhub/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
df['lm'] = df['extr'].progress_apply(summarize_text)

100%|██████████| 17685/17685 [10:43<00:00, 27.48it/s]
